In [1]:
!pip install autogen-agentchat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install playwright

In [2]:
!python -m playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 14.0 MB/s eta 0:00:00


In [3]:
!pip install  beautifulsoup4

In [4]:
!pip install  chromium

In [5]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.2/859.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
import asyncio
import json
import time
from typing import Dict, List, Any
from datetime import datetime
import pandas as pd
from playwright.async_api import async_playwright
import autogen
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# Configuration
config_list = [
    {
        "model": "gpt-4",
        "api_key": your_api_key,  # Replace with your actual API key
        "base_url": "https://api.openai.com/v1",
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0.1,
    "timeout": 120,
}

In [9]:
class SimpleWebScraper:
    """Simple web scraper using Playwright"""

    def __init__(self):
        self.browser = None
        self.context = None
        self.results = []

    async def setup_browser(self, headless=True):
        """Initialize browser"""
        playwright = await async_playwright().start()
        self.browser = await playwright.chromium.launch(headless=headless)
        self.context = await self.browser.new_context(
            user_agent="Mozilla/5.0 (compatible; DataBot/1.0)",
            viewport={"width": 1920, "height": 1080}
        )

    async def scrape_page(self, url: str, selectors: Dict[str, str]) -> Dict[str, Any]:
        """Scrape a single page"""
        try:
            page = await self.context.new_page()
            await page.goto(url, wait_until="networkidle", timeout=30000)

            data = {"url": url, "scraped_at": datetime.now().isoformat()}

            for key, selector in selectors.items():
                try:
                    elements = await page.query_selector_all(selector)
                    data[key] = [await el.text_content() for el in elements if await el.text_content()]
                except Exception as e:
                    data[key] = []
                    print(f"Error scraping {key}: {e}")

            await page.close()
            return data

        except Exception as e:
            print(f"Error scraping {url}: {e}")
            return {"url": url, "error": str(e)}

    async def close(self):
        """Close browser"""
        if self.browser:
            await self.browser.close()

In [10]:
# Global scraper instance
scraper = SimpleWebScraper()

In [92]:
class MultiAgentWebScrapingSystem:
    """5-Agent Web Scraping System"""

    def __init__(self):
        self.results = []
        self.session_data = {}
        self.setup_agents()

    def setup_agents(self):
        """Create the 5 specialized agents"""

        # 1. Master Coordinator Agent
        self.coordinator = AssistantAgent(
            name="MasterCoordinator",
            system_message="""You are the Master Coordinator Agent. Your role is to:
            1. Analyze user queries and understand what data needs to be scraped
            2. Coordinate with other agents to plan and execute scraping tasks
            3. Compile and present final results
            4. Always start by asking the SearchStrategy agent to analyze the query
            5. Distribute tasks to appropriate scraper agents
            6. Summarize results from all agents

            Format your responses clearly and coordinate the workflow efficiently.""",
            llm_config=llm_config
        )

        # 2. Search Strategy Agent
        self.strategy_agent = AssistantAgent(
            name="SearchStrategy",
            system_message="""You are the Search Strategy Agent. Your role is to:
            1. Analyze user queries to understand intent and extract keywords
            2. Identify relevant websites and data sources for scraping
            3. Generate specific scraping strategies for different types of content
            4. Recommend which specialized scraper agents should handle each task
            5. Suggest CSS selectors and scraping patterns

            Provide detailed scraping plans with:
            - Target websites
            - Recommended selectors
            - Data extraction strategies
            - Which scraper agent to use""",
            llm_config=llm_config
        )

        # 3. Generic Web Scraper Agent
        self.generic_scraper = AssistantAgent(
            name="GenericScraper",
            system_message="""You are the Generic Web Scraper Agent. Your role is to:
            1. Handle general-purpose web scraping tasks
            2. Extract content from various website types
            3. Use Playwright to navigate and scrape web pages
            4. Report scraping results and any issues encountered

            When given a scraping task, provide:
            - Confirmation of the task received
            - Scraping approach and selectors to use
            - Results summary after scraping""",
            llm_config=llm_config
        )

        # 4. Data Processing Agent
        self.data_processor = AssistantAgent(
            name="DataProcessor",
            system_message="""You are the Data Processing Agent. Your role is to:
            1. Clean and structure scraped data
            2. Remove duplicates and invalid entries
            3. Standardize data formats
            4. Organize data into structured formats
            5. Perform basic data validation

            When processing data, provide:
            - Data cleaning summary
            - Structure improvements made
            - Final processed data format
            - Data quality assessment""",
            llm_config=llm_config
        )

        # 5. Quality Assurance Agent
        self.qa_agent = AssistantAgent(
            name="QualityAssurance",
            system_message="""You are the Quality Assurance Agent. Your role is to:
            1. Verify data accuracy and completeness
            2. Check for inconsistencies across scraped data
            3. Validate data quality metrics
            4. Flag potential errors or issues
            5. Provide quality scores and recommendations

            When reviewing data, provide:
            - Quality assessment report
            - Identified issues and inconsistencies
            - Recommendations for improvement
            - Overall quality score (1-10)""",
            llm_config=llm_config
        )

        # User proxy for human interaction
        self.user_proxy = UserProxyAgent(
            name="UserProxy",
            system_message="You facilitate communication between the user and the agent system.",
            code_execution_config={"work_dir": "scraping_work", "use_docker": False},
            human_input_mode="NEVER"
        )

        # Create group chat
        self.agents = [
            self.coordinator,
            self.strategy_agent,
            self.generic_scraper,
            self.data_processor,
            self.qa_agent
        ]

        self.groupchat = GroupChat(
            agents=self.agents,
            messages=[],
            max_round=15,
            speaker_selection_method="round_robin"
        )

        self.manager = GroupChatManager(
            groupchat=self.groupchat,
            llm_config=llm_config
        )

    async def execute_scraping_task(self, query: str):
        """Execute the multi-agent scraping workflow"""

        print(f" Starting multi-agent scraping for query: '{query}'")
        print("="*60)

        # Start the conversation
        initial_message = f"""
        New scraping request: "{query}"

        Please coordinate to:
        1. Analyze the query and determine scraping strategy
        2. Identify target websites and data sources
        3. Execute scraping tasks
        4. Process and clean the data
        5. Perform quality assurance
        6. Provide final results

        Let's begin with the SearchStrategy agent analyzing this query.
        """

        # Execute the group chat
        await self.user_proxy.a_initiate_chat(
            self.manager,
            message=initial_message
        )

        return self.groupchat.messages

    async def scrape_with_agents(self, query: str, urls: List[str] = None, selectors: Dict[str, str] = None):
        """Simplified scraping function for demo purposes"""

        if not urls:
            # Default example URLs for demonstration
            urls = [
                "https://example.com",
                "https://httpbin.org/html"
            ]

        if not selectors:
            # Default selectors
            selectors = {
                "title": "title",
                "headings": "h1, h2, h3",
                "paragraphs": "p",
                "links": "a",
                "code_blocks": "pre, code"
            }

        # Setup browser
        await scraper.setup_browser()

        results = []
        for url in urls:
            print(f"📄 Scraping: {url}")
            data = await scraper.scrape_page(url, selectors)
            results.append(data)

        await scraper.close()
        return results


In [12]:

def create_custom_scraping_session(query: str, urls: List[str], selectors: Dict[str, str]):
    """Create a custom scraping session"""

    async def run_custom_session():
        system = MultiAgentWebScrapingSystem()

        # First, let agents analyze the query
        messages = await system.execute_scraping_task(query)

        # Then perform actual scraping
        scraping_results = await system.scrape_with_agents(query, urls, selectors)

        return {
            "query": query,
            "agent_discussion": messages,
            "scraping_results": scraping_results,
            "timestamp": datetime.now().isoformat()
        }

    return run_custom_session()


In [94]:
# # Custom scraping session
urls = ["https://medium.com/@joerosborne/intro-to-web-scraping-build-your-first-scraper-in-5-minutes-1c36b5c4b110"]
selectors = {"title": "title", "headings": "h1, h2, h3", "paragraphs": "p","code_blocks": "pre, code"}

result = await create_custom_scraping_session("scrap content on page",selectors=selectors,urls=urls)

🚀 Starting multi-agent scraping for query: 'scrap content on page'
UserProxy (to chat_manager):


        New scraping request: "scrap content on page"
        
        Please coordinate to:
        1. Analyze the query and determine scraping strategy
        2. Identify target websites and data sources
        3. Execute scraping tasks
        4. Process and clean the data
        5. Perform quality assurance
        6. Provide final results
        
        Let's begin with the SearchStrategy agent analyzing this query.
        

--------------------------------------------------------------------------------

Next speaker: MasterCoordinator

MasterCoordinator (to chat_manager):

SearchStrategy Agent, could you please analyze the user's query and propose a suitable scraping strategy? The user wants to scrape content from a page but hasn't specified which page or what kind of content. We may need to ask for more details.

---------------------------------------------------------------

171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 21.9s171.6 MiB [] 0% 13.5s171.6 MiB [] 0% 7.2s171.6 MiB [] 1% 4.5s171.6 MiB [] 2% 3.9s171.6 MiB [] 2% 3.2s171.6 MiB [] 3% 2.8s171.6 MiB [] 4% 2.7s171.6 MiB [] 5% 2.9s171.6 MiB [] 5% 2.8s171.6 MiB [] 6% 2.8s171.6 MiB [] 7% 2.7s171.6 MiB [] 8% 2.6s171.6 MiB [] 9% 2.5s171.6 MiB [] 10% 2.6s171.6 MiB [] 11% 2.5s171.6 MiB [] 12% 2.4s171.6 MiB [] 13% 2.3s171.6 MiB [] 14% 2.1s171.6 MiB [] 15% 2.1s171.6 MiB [] 16% 2.0s171.6 MiB [] 17% 2.0s171.6 MiB [] 18% 1.9s171.6 MiB [] 19% 1.9s171.6 MiB [] 20% 1.9s171.6 MiB [] 20% 2.0s171.6 MiB [] 21% 2.0s171.6 MiB [] 22% 2.0s171.6 MiB [] 23% 1.9s171.6 MiB [] 24% 1.9s171.6 MiB [] 25% 1.9s171.6 MiB [] 26% 1.9s171.6 MiB [] 26% 1.8s171.6 MiB [] 27% 1.8s171.6 MiB [] 28% 1.8s171.6 MiB [] 29% 1.8s171.6 MiB [] 30% 1.7s171.6 MiB [] 31% 1.7s171.6 MiB [] 32% 1.6s171.6 MiB [] 33% 1.7s171.6 MiB [] 33% 1.6s171.6 MiB [] 34% 1.6s171.6 MiB [] 35% 1.6s171.6 MiB [] 36% 1.5s171.6 MiB [] 37% 1.5s171.6 MiB [] 38% 1.5s171.6 MiB [] 39% 1.5s171.6

In [95]:
print(result)

{'query': 'scrap content on page', 'agent_discussion': [{'content': '\n        New scraping request: "scrap content on page"\n        \n        Please coordinate to:\n        1. Analyze the query and determine scraping strategy\n        2. Identify target websites and data sources\n        3. Execute scraping tasks\n        4. Process and clean the data\n        5. Perform quality assurance\n        6. Provide final results\n        \n        Let\'s begin with the SearchStrategy agent analyzing this query.\n        ', 'role': 'user', 'name': 'UserProxy'}, {'content': "SearchStrategy Agent, could you please analyze the user's query and propose a suitable scraping strategy? The user wants to scrape content from a page but hasn't specified which page or what kind of content. We may need to ask for more details.", 'role': 'user', 'name': 'MasterCoordinator'}, {'content': "Indeed, the user's query is quite broad and lacks specific details. To formulate an effective scraping strategy, we nee

In [96]:
for res in result:
  print(res)
  import time
  time.sleep(1)

query
agent_discussion
scraping_results
timestamp


In [97]:
result['scraping_results']

[{'url': 'https://medium.com/@joerosborne/intro-to-web-scraping-build-your-first-scraper-in-5-minutes-1c36b5c4b110',
  'scraped_at': '2025-07-15T13:59:41.931227',
  'title': ['Intro to Web Scraping: Build Your First Scraper in 5 Minutes | by Joe Osborne | Medium'],
  'headings': ['Intro to Web Scraping: Build Your First Scraper in 5 Minutes',
   'A quick guide to help you build a simple web scraper.',
   'Step 1',
   'Step 2',
   'Step 3',
   'Resources',
   'Sign up to discover human stories that deepen your understanding of the world.',
   'Free',
   'Membership',
   'Written by Joe Osborne',
   'Responses (9)',
   'More from Joe Osborne',
   'How to Log Every Request and Response in FastAPI',
   '📕 A guide on efficiently storing logs for every request and response in your FastAPI app',
   'FastAPI with API Key Authentication',
   '📘 How to set up your FastAPI app with a simple API key auth system.',
   'Easy Web Scraping: Scrape an Entire Website in Under 10 Minutes',
   '📕 A simple

In [80]:
import json

with open("result.txt", "w", encoding="utf-8") as f:
    for item in result['scraping_results']:
        f.write(json.dumps(item, ensure_ascii=False, indent=2))
        f.write("\n\n")

In [51]:
df=analyze_results(result['scraping_results'])

In [55]:
result['scraping_results']

[{'url': 'https://medium.com/@joerosborne/intro-to-web-scraping-build-your-first-scraper-in-5-minutes-1c36b5c4b110',
  'scraped_at': '2025-07-15T13:09:57.982084',
  'title': ['Intro to Web Scraping: Build Your First Scraper in 5 Minutes | by Joe Osborne | Medium'],
  'headings': ['Intro to Web Scraping: Build Your First Scraper in 5 Minutes',
   'A quick guide to help you build a simple web scraper.',
   'Step 1',
   'Step 2',
   'Step 3',
   'Resources',
   'Written by Joe Osborne',
   'Responses (9)',
   'More from Joe Osborne',
   'How to Log Every Request and Response in FastAPI',
   '📕 A guide on efficiently storing logs for every request and response in your FastAPI app',
   'FastAPI with API Key Authentication',
   '📘 How to set up your FastAPI app with a simple API key auth system.',
   'Easy Web Scraping: Scrape an Entire Website in Under 10 Minutes',
   '📕 A simple guide on building and running a web scraper.',
   'Guide: How to Build a Customized AI Assistant using OpenAI’s 

In [65]:
import pandas as pd
from typing import List, Dict, Any


def analyze_results(results: List[Dict[str, Any]]) -> pd.DataFrame:
    """Convert scraping results to DataFrame including raw content"""

    if not results:
        return pd.DataFrame()

    flattened = []
    for result in results:
        if 'error' not in result:
            flattened.append({
                'url': result.get('url', ''),
                'scraped_at': result.get('scraped_at', ''),
                'title_count': len(result.get('title', [])),
                'headings_count': len(result.get('headings', [])),
                'paragraphs_count': len(result.get('paragraphs', [])),
                'links_count': len(result.get('links', [])) if 'links' in result else 0,
                # Join lists into plain text strings for CSV
                'title': " | ".join(result.get('title', [])),
                'headings': " | ".join(result.get('headings', [])),
                'paragraphs': " | ".join(result.get('paragraphs', []))
            })

    return pd.DataFrame(flattened)



In [66]:
# Assume `scraped_data` is your list of dicts
df = analyze_results(result['scraping_results'])

In [59]:
# Export to CSV
df.to_csv('scraping_analysis.csv', index=False)
print("✅ CSV file saved as 'scraping_analysis.csv'")

✅ CSV file saved as 'scraping_analysis.csv'


In [62]:
pd.read_csv('/content/scraping_analysis.csv')

,url,title_count,headings_count,paragraphs_count,links_count,scraped_at
0,https://medium.com/@joerosborne/intro-to-web-s...,1,30,86,0,2025-07-15T13:09:57.982084
